In [108]:
import pandas as pd
import os
import re
import datetime
import time

In [109]:
folder = r"240310_Nax"

In [110]:
# patterns_heal = [ # As of Python version 3.7, dictionaries are ordered
#     ["HEALEDCRITOTHEROTHER", "%s's %s critically heals %s for %d.", (source, attack, target, value)],
#     "HEALEDCRITOTHERSELF": "%s's %s critically heals you for %d.",
#     "HEALEDCRITSELFOTHER": "Your %s critically heals %s for %d.",
#     "HEALEDCRITSELFSELF": "Your %s critically heals you for %d.",
#     "HEALEDOTHEROTHER": "%s's %s heals %s for %d.",
#     "HEALEDOTHERSELF": "%s's %s heals you for %d.",
#     "HEALEDSELFOTHER": "Your %s heals %s for %d.",
#     "HEALEDSELFSELF": "Your %s heals you for %d.",
# ]
patterns_aura = [
    ["AURAADDEDOTHERHARMFUL", "%s is afflicted by %s.", ("target", "attack")],
    ["AURAADDEDOTHERHELPFUL", "%s gains %s.", ("target", "attack")],
    ["AURAADDEDSELFHARMFUL", "You are afflicted by %s.", ("attack")],
    ["AURAADDEDSELFHELPFUL", "You gain %s.", ("attack")]
]

In [111]:
def PreparePattern(pattern):
    pattern = pattern.replace("%s", "(.+)")
    pattern = pattern.replace("%d", "(\d+)")
    return pattern

In [112]:
def ParseTimeStamp(line):
    pattern = r"(\d+)/(\d+) (\d+):(\d+):(\d+).(\d+)  (.*)"
    match = re.search(pattern, line)
    year = 2024
    month = int(match.group(1))
    day = int(match.group(2))
    hour = int(match.group(3))
    minute = int(match.group(4))
    second = int(match.group(5))
    millisecond = int(match.group(6))
    rem = match.group(7)
    date_time = datetime.datetime(year,month,day,hour,minute,second)
    timestamp = time.mktime(date_time.timetuple())+millisecond/1000
    
    return rem, timestamp

In [134]:
def ParseCombatLog(line, pattern_data):
    line_data = {
        "source": None,
        "target": None,
        "attack": None,
        "value": None
    }
    for pattern_dict in pattern_data:
        pattern = pattern_dict[1]
        pattern = PreparePattern(pattern)
        match = re.search(pattern, line)
        if match:
            for idx, kind in enumerate(pattern_dict[2]):
                line_data[kind] = match.group(idx+1)
            return line_data
    print("### Couldnt find matching pattern for: ", line)
    return line_data

In [135]:
filepaths = [os.path.join(folder, f) for f in os.listdir(folder)]
df_auras = pd.DataFrame()
for filepath in filepaths:
    print("Reading ", filepath)
    file = open(filepath, 'r')
    lines = file.readlines()
    cnt = 0
    for line in lines:
        print(line)
        rem, timestamp = ParseTimeStamp(line)
        print(timestamp)
        data = ParseCombatLog(rem, patterns_aura)
        print(data)
        cnt = cnt+1
        if cnt > 10:
            break
    break

Reading  240310_Nax\WoWCombatLog_Kikidora.txt
3/10 19:02:35.473  Pestilentia gains Greater Blessing of Salvation.

1710093755.473
{'source': None, 'target': 'Pestilentia', 'attack': 'Greater Blessing of Salvation', 'value': None}
3/10 19:02:35.473  Baldnic gains Greater Blessing of Salvation.

1710093755.473
{'source': None, 'target': 'Baldnic', 'attack': 'Greater Blessing of Salvation', 'value': None}
3/10 19:02:35.475  Receptor gains Greater Blessing of Salvation.

1710093755.475
{'source': None, 'target': 'Receptor', 'attack': 'Greater Blessing of Salvation', 'value': None}
3/10 19:02:35.783  Aspect of the Cheetah fades from Whiskeytango.

1710093755.783
### Couldnt find matching pattern for:  Aspect of the Cheetah fades from Whiskeytango.
{'source': None, 'target': None, 'attack': None, 'value': None}
3/10 19:02:35.783  Whiskeytango gains Aspect of the Hawk.

1710093755.783
{'source': None, 'target': 'Whiskeytango', 'attack': 'Aspect of the Hawk', 'value': None}
3/10 19:02:36.336  